In [47]:
import numpy as np
import keras
import tensorflow as tf
import csv
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
from sklearn.model_selection import train_test_split

lines = []
with open('data_CarND/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(csvfile)
    for line in reader:
        lines.append(line)
        
train_samples, validation_samples = train_test_split(lines, test_size=0.2, random_state=42)

In [49]:
from sklearn.utils import shuffle 

def generator(samples, batch_size = 32, correction = 0.2):
    num_samples = len(samples)
    while 1:
        samples = shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset: offset + batch_size]
            
            images = []
            angles = []
            for batch_sample in batch_samples:
                #Center image
                name = batch_sample[0].split('/')[-1]
                current_path = 'data_CarND/IMG/' + name
                center_image = cv2.imread(current_path)
                center_image = cv2.cvtColor(center_image, cv2.COLOR_BGR2RGB)
                center_image = cv2.resize(center_image, (160, 80))
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                
                #Left image
                name = batch_sample[1].split('/')[-1]
                current_path = 'data_CarND/IMG/' + name
                left_image = cv2.imread(current_path)
                left_image = cv2.cvtColor(left_image, cv2.COLOR_BGR2RGB)
                left_image = cv2.resize(left_image, (160, 80))
                left_angle = center_angle + correction
                images.append(left_image)
                angles.append(left_angle)
                
                #Right image
                name = batch_sample[2].split('/')[-1]
                current_path = 'data_CarND/IMG/' + name
                right_image = cv2.imread(current_path)
                right_image = cv2.cvtColor(right_image, cv2.COLOR_BGR2RGB)
                right_image = cv2.resize(right_image, (160, 80))
                right_angle = center_angle - correction
                images.append(right_image)
                angles.append(right_angle)
            
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)

In [50]:
train_generator = generator(train_samples, batch_size=32, correction=0.2)
validation_generator = generator(validation_samples, batch_size=32, correction=0.2)

row, col, ch = 160, 320, 3

In [51]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Lambda, Reshape, Cropping2D, Conv2D

model = Sequential()
model.add(Cropping2D(cropping=((25, 12), (0, 0)), input_shape=(80, 160, 3)))
model.add(Lambda(lambda x: x / 255.0 - 0.5))
model.add(Conv2D(filters=24, kernel_size=5, strides=(2, 2), 
                 padding='same', activation='relu'))
model.add(Conv2D(filters=36, kernel_size=5, strides=(2, 2),
                 padding='same', activation='relu'))
model.add(Conv2D(filters=48, kernel_size=5, strides=(2, 2),
                 padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, strides=(1, 1),
                 padding='valid', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, strides=(1, 1),
                 padding='valid', activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.summary()

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, steps_per_epoch=len(train_samples),
                    validation_data=validation_generator,
                    validation_steps=len(validation_samples),
                    epochs=1, verbose=1)

model.save('jupyter_new_model.h5')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_17 (Cropping2D)   (None, 43, 160, 3)        0         
_________________________________________________________________
lambda_17 (Lambda)           (None, 43, 160, 3)        0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 22, 80, 24)        1824      
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 11, 40, 36)        21636     
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 6, 20, 48)         43248     
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 4, 18, 64)         27712     
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 2, 16, 64)         36928     
__________

In [72]:
from keras.models import load_model

teste = load_model('jupyter_new_model.h5')

In [73]:
teste.compile(loss='mse', optimizer='adam')
teste.fit_generator(train_generator, steps_per_epoch=len(train_samples),
                    validation_data=validation_generator,
                    validation_steps=len(validation_samples),
                    epochs=2, verbose=1)

hist = teste.save('jupyter_new_model_v2.h5')

Epoch 1/2
10556/10556 [==============================] - 4254s 403ms/step - loss: 6.2039e-04 - val_loss: 0.0071
Epoch 2/2
 1416/10556 [===>..........................] - ETA: 59:57 - loss: 4.1483e-04

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pedrohrpbs/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-73-9eb38f5889d4>", line 5, in <module>
    epochs=2, verbose=1)
  File "/home/pedrohrpbs/anaconda3/lib/python3.6/site-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/home/pedrohrpbs/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1418, in fit_generator
    initial_epoch=initial_epoch)
  File "/home/pedrohrpbs/anaconda3/lib/python3.6/site-packages/keras/engine/training_generator.py", line 181, in fit_generator
    generator_output = next(output_generator)
  File "/home/pedrohrpbs/anaconda3/lib/python3.6/site-packages/keras/utils/data_utils.py", line 685, in get
    inputs = self.queue.get(block=True).get()
  File "/home/pedrohrpbs/anaconda3/lib/python3.6/multiprocessing/pool.py", line 

KeyboardInterrupt: 